In [1]:
from beamngpy import BeamNGpy, Scenario, Vehicle, Road, ScenarioObject
from beamngpy.sensors import Damage
import numpy as np
from time import sleep, time

# Collision Avoidance

## Setting up Simulation Parameters 

In [2]:
beamng = BeamNGpy('localhost', 64256, user='D:/ros', home='D:/tech.0.21.x')
bng = beamng.open(launch=False)
bng.set_deterministic()

## Scenario Definition

In [3]:
def launch_scenario():
    scenario = Scenario('smallgrid', 'Intersection')

    road_a = Road('track_editor_C_center', rid='road_a')
    road_a.nodes.extend([(0, 0, 0, 5),
                        (100, 0, 0, 5)])
    road_b = Road('track_editor_C_center', rid='road_b')
    road_b.nodes.extend([(50, -50, 0, 5),
                        (50, 50, 0, 5)])
    scenario.add_road(road_a)
    scenario.add_road(road_b)

    ego = Vehicle('ego', model='etk800', color='Green')
    pos = (0, 0, 0)
    rot = (0, 0, -.707, .707)
    damage = Damage()
    ego.attach_sensor('damage', damage)
    scenario.add_vehicle(ego, pos=pos, rot=None, rot_quat=rot)

    obstacle = Vehicle('obstacle', model='etk800')
    damage_o = Damage()
    obstacle.attach_sensor('damage', damage_o)
    pos = (50, -50, 0)
    rot = (0, 0, 1, 0)
    scenario.add_vehicle(obstacle, pos=pos, rot=None, rot_quat=rot)
    scenario.make(bng)
    bng.load_scenario(scenario)
    bng.start_scenario()
    bng.switch_vehicle(ego)
    bng.pause()
    return obstacle, ego


## Defining Test Oracle

In [23]:
def test(set_up_ai):
    obstacle, ego = launch_scenario()
    set_up_ai(ego, obstacle)
    bng.resume()
    sleep(30)
    bng.pause()
    ego.poll_sensors()
    obstacle.poll_sensors()
    ego_damage = ego.sensors['damage'].data['damage']
    ego_pos = np.array(ego.sensors['state'].data['pos'])
    ego_goal_pos = np.array([100, 0, 0])

    obst_damage = obstacle.sensors['damage'].data['damage']
    obst_pos = np.array(obstacle.sensors['state'].data['pos'])
    obst_goal_pos = np.array([50, 50, 0])
    epsilon = 1
    test_damage = ego_damage > epsilon or obst_damage > epsilon
    print(f'obst_pos: {obst_pos}')
    print(f'obst_goal_pos:{obst_goal_pos}')
    print(f'ego_pos: {ego_pos}')
    print(f'ego_goal_pos:{ego_goal_pos}')
    test_goal_pos = ((ego_pos - ego_goal_pos) > epsilon).any() or ((obst_pos - obst_goal_pos) > epsilon).any()
    if test_damage or test_goal_pos:
        print('failed')
    else:
        print('passed')           


### Executing Tests

In [25]:
def wp1(ego, obstacle):
    ego_script = [{'x':50, 'y':0, 'z':0, 't':10},
              {'x':100, 'y':0, 'z':0, 't':20}, {'x':100, 'y':0, 'z':0, 't':21}]
    ego.ai_set_script(ego_script)

    obst_script = [{'x':50, 'y':0, 'z':0, 't':15},
                   {'x':50, 'y':50, 'z':0, 't':25}, {'x':50, 'y':50, 'z':0, 't':29}]
    obstacle.ai_set_script(obst_script)

def no_crash():
    test(wp1)

no_crash()

obst_pos: [50.19941711 27.67672348  0.20447171]
obst_goal_pos:[50 50  0]
ego_pos: [91.70462799 -0.58370495  0.20409746]
ego_goal_pos:[100   0   0]
passed


In [29]:
def wp2(ego, obstacle):
    ego_script = [{'x':50, 'y':0, 'z':0, 't':20},
              {'x':100, 'y':0, 'z':0, 't':25}, {'x':100, 'y':0, 'z':0, 't':30}]
    ego.ai_set_script(ego_script)

    obst_script = [{'x':50, 'y':0, 'z':0, 't':20},
                   {'x':50, 'y':75, 'z':0, 't':25}, {'x':50, 'y':50, 'z':0, 't':30}]
    obstacle.ai_set_script(obst_script)

def crash():
    test(wp2)

crash()

obst_pos: [50.52590942 65.46222687  0.20163694]
obst_goal_pos:[50 50  0]
ego_pos: [97.89096832 -0.51444983  0.20304944]
ego_goal_pos:[100   0   0]
failed
